In [ ]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
data = '../data/Left1.csv'

In [3]:
df = pd.read_csv(data, index_col = 0)

In [4]:
df.head()

,Source,Serial,City,Station,Latitude,Longitude,Datetime,PM,USAQI,CO2,Temperature,RelativeHumidity,weekday,hour,month,year,dayofmonth
0,EPD,NaN,Lahore,Quaid-e-azam Township,NaN,NaN,2014-07-31 00:00:00,94.335,NaN,NaN,NaN,NaN,3,0,7,2014,31
1,EPD,NaN,Lahore,Town Hall,NaN,NaN,2014-07-31 00:00:00,93.620,NaN,NaN,NaN,NaN,3,0,7,2014,31
2,EPD,NaN,Lahore,Town Hall,NaN,NaN,2014-08-31 00:00:00,57.700,NaN,NaN,NaN,NaN,6,0,8,2014,31
3,EPD,NaN,Lahore,Quaid-e-azam Township,NaN,NaN,2014-09-30 00:00:00,151.000,NaN,NaN,NaN,NaN,1,0,9,2014,30
4,EPD,NaN,Lahore,Town Hall,NaN,NaN,2014-09-30 00:00:00,49.540,NaN,NaN,NaN,NaN,1,0,9,2014,30


In [5]:
df_new = df.drop(['Source', 'Serial', 'Station', 'Latitude', 'Longitude', 'Datetime'], axis = 1)

In [6]:
print(df_new.isna().mean())
print(len(df_new))
print(df_new.City.unique())

df_ready = df_new.dropna()
print(len(df_ready))
print(df_ready.City.unique())

City                0.000000
PM                  0.000000
USAQI               0.003970
CO2                 0.521804
Temperature         0.123743
RelativeHumidity    0.123623
weekday             0.000000
hour                0.000000
month               0.000000
year                0.000000
dayofmonth          0.000000
dtype: float64
391917
['Lahore' 'Karachi' 'Islamabad' 'Peshawar' 'Chakwal' 'Faisalabad'
 'Gujranwala' 'Bahawalpur' 'Rawalpindi' 'Muridke' 'Raiwind' 'Diplo'
 'Sialkot']
187340
['Karachi' 'Islamabad' 'Lahore' 'Peshawar' 'Bahawalpur' 'Diplo']


In [7]:
df_ready.head()

,City,PM,USAQI,CO2,Temperature,RelativeHumidity,weekday,hour,month,year,dayofmonth
14,Karachi,24.0,76.0,502.0,30.0,78.0,3,14,10,2016,6
15,Karachi,22.0,72.0,420.0,30.0,76.0,3,15,10,2016,6
16,Karachi,21.0,70.0,416.0,30.0,76.0,3,17,10,2016,6
17,Karachi,22.0,72.0,416.0,28.0,80.0,3,18,10,2016,6
18,Karachi,23.0,74.0,415.0,28.0,80.0,3,19,10,2016,6


In [8]:
def transform(df):
    df = (df - df.mean())/df.std()
    return df

In [9]:
def createDataSets(df, city, frac):
    
    df_part = df[df.City != city]
    indexes = list(df_part.index)
    random.shuffle(indexes)
    length = int(len(indexes) * frac)
    take = indexes[:length]
    
    df = df.drop('City', axis = 1)
    df = transform(df)
    
    df_train = df.loc[take]
    df_test = df.drop(take)
    return df_train, df_test
    


In [10]:
def createXandY(df):
    Y = df['CO2']
    X = df.drop(['CO2'], axis = 1)
    return nd.array(X.values), nd.array(Y.values)

In [11]:
means = df_ready.drop('City' , axis = 1).mean()
stds = df_ready.drop('City' , axis = 1).std()

df_train, df_test = createDataSets(df_ready, 'Islamabad', 0.7)
X_train, Y_train = createXandY(df_train)
X_test, Y_test = createXandY(df_test)

print(len(X_test)/ (len(X_test) + len(X_train)))

0.34038112522686026


In [12]:
in_shape = X_train[0].shape
model = Sequential()
model.add(Dense(10, activation = 'sigmoid', input_shape = in_shape))
model.add(Dense(1, activation = 'linear'))

model.summary()

In [13]:
model.compile(optimizer = 'sgd', loss = 'mean_squared_error')

In [14]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 128, verbose = True)